# Scraping meeting minutes since the start of 2019 from the Berkeley Zoning Adjustments Board

In [ ]:
import os
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE_URL = "https://berkeleyca.gov"
PAGE_URL_TEMPLATE = "https://berkeleyca.gov/your-government/boards-commissions/zoning-adjustments-board?page={}"
DOWNLOAD_DIR = "zab_minutes_pdfs"
NUM_PAGES = 9  # pages 0 through 8

# Create directory if it doesn't exist
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

for page in range(NUM_PAGES):
    print(f"\nScraping page {page + 1} of {NUM_PAGES}")
    url = PAGE_URL_TEMPLATE.format(page)
    try:
        resp = requests.get(url)
        resp.raise_for_status()
    except Exception as e:
        print(f"Failed to fetch page {page}: {e}")
        continue

    soup = BeautifulSoup(resp.content, "html.parser")
    rows = soup.find_all("tr", class_="table-expand-row-content")

    found_any = False
    for row in rows:
        minutes_links = row.find_all("a", string="Minutes")
        for link in minutes_links:
            relative_url = link.get("href")
            if relative_url:
                full_url = urljoin(BASE_URL, relative_url)
                filename = os.path.basename(relative_url.split("?")[0])
                save_path = os.path.join(DOWNLOAD_DIR, filename)

                print(f"Downloading {filename}")
                try:
                    pdf_resp = requests.get(full_url)
                    pdf_resp.raise_for_status()
                    with open(save_path, "wb") as f:
                        f.write(pdf_resp.content)
                    found_any = True
                    time.sleep(1)  # being polite to the server
                except Exception as e:
                    print(f"Error downloading {full_url}: {e}")
    if not found_any:
        print("No 'Minutes' PDFs found on this page.")
